In [4]:
#Pquetería
import pandas as pd
import re
#import networkx as nx
#import graph_tool.all as gt
import numpy as np
#import geopandas as gpd  
import matplotlib.pyplot as plt

Matplotlib is building the font cache; this may take a moment.


In [5]:
#Bases de datos
url='https://raw.githubusercontent.com/DianaMCV/Redes/main/comercio_2019.csv'
url1='https://raw.githubusercontent.com/DianaMCV/Redes/main/comercio_2020.csv'
url2='https://raw.githubusercontent.com/DianaMCV/Redes/main/comercio_2021.csv'
url3='https://raw.githubusercontent.com/RBESPINOSA/proyecto_redes/main/coordenadas.csv'
url4='https://raw.githubusercontent.com/RBESPINOSA/proyecto_redes/main/distancias_final.csv'

c2019 = pd.read_csv(url, index_col=0, encoding='latin-1') 
#c2020 = pd.read_csv(url1, index_col=0, encoding='latin-1') 
#c2021 = pd.read_csv(url2, index_col=0, encoding='latin-1') 
coordenadas = pd.read_csv(url3, encoding='latin-1') 
#distancias = pd.read_csv(url4, encoding='latin-1')

In [6]:
c2019.columns = c2019.columns.str.lower()
c2019['origen']=c2019['origen'].str.lower()
c2019['destino']=c2019['destino'].str.lower()
c2019['destino']=c2019['destino'].apply(lambda x:(re.search('(.+(?=:))', x)).group(1))
c2019.head()

,producto,calidad,presentación,origen,destino,precio mín,precio max,precio frec,obs.
Fecha,,,,,,,,,
22/04/2019,Aguacate Hass,Primera,Caja de 15 kg.,aguascalientes,nayarit,808,810,809,NaN
23/04/2019,Aguacate Hass,Primera,Caja de 15 kg.,aguascalientes,nayarit,808,810,809,NaN
24/04/2019,Aguacate Hass,Primera,Caja de 15 kg.,aguascalientes,nayarit,808,810,809,NaN
25/04/2019,Aguacate Hass,Primera,Caja de 15 kg.,aguascalientes,nayarit,808,810,809,NaN
26/04/2019,Aguacate Hass,Primera,Caja de 15 kg.,aguascalientes,nayarit,808,810,809,NaN


In [7]:
c2019.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14997 entries, 22/04/2019 to 29/04/2019
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   producto      14997 non-null  object
 1   calidad       14997 non-null  object
 2   presentación  14997 non-null  object
 3   origen        14997 non-null  object
 4   destino       14997 non-null  object
 5   precio mín    14997 non-null  object
 6   precio max    14997 non-null  object
 7   precio frec   14997 non-null  object
 8   obs.          764 non-null    object
dtypes: object(9)
memory usage: 1.1+ MB


In [8]:
#agrupamos por la característica
adj_2019 = c2019.groupby(['origen', 'destino']).size().rename('frecuencia').reset_index()


#Eliminamos self-loops
adj_2019=adj_2019[adj_2019['origen']!=adj_2019['destino']]
adj_2019.head()

,origen,destino,frecuencia
1,aguascalientes,chihuahua,6
2,aguascalientes,durango,24
3,aguascalientes,michoacán,12
4,aguascalientes,nayarit,18
5,aguascalientes,tamaulipas,6


In [9]:
#eliminamos el origen nacional porque no le veo interpretación gráfica

adj_2019.drop(adj_2019.loc[adj_2019['origen']=='nacional'].index, inplace=True)

In [11]:
def to_id(nombre):
    if nombre == 'aguascalientes':
        return 1
    if nombre == 'baja california':
        return 2
    if nombre == 'baja california sur':
        return 3
    if nombre == 'campeche':
        return 4
    if nombre == 'chiapas': 
        return 5
    if nombre == 'chihuahua': 
        return 6
    if nombre == 'coahuila': 
        return 7
    if nombre == 'colima':
        return 8
    if nombre == 'distrito federal': 
        return 9
    if nombre == 'durango':
        return 10
    if nombre == 'guanajuato':
        return 11
    if nombre == 'guerrero':
        return 12
    if nombre == 'hidalgo': 
        return 13
    if nombre == 'jalisco':
        return 14
    if nombre == 'méxico':
        return 15
    if nombre == 'michoacán':
        return 16
    if nombre == 'morelos':
        return 17        
    if nombre == 'nayarit':
        return 18
    if nombre == 'nuevo león':
        return 19
    if nombre == 'oaxaca':
        return 20
    if nombre == 'puebla':
        return 21
    if nombre == 'querétaro':
        return 22
    if nombre == 'quintana roo':
        return 23
    if nombre == 'san luis potosí':
        return 24
    if nombre == 'sinaloa':
        return 25
    if nombre == 'sonora':
        return 26
    if nombre == 'tabasco':
        return 27
    if nombre == 'tamaulipas':
        return 28
    if nombre == 'veracruz':
        return 29
    if nombre == 'yucatán':
        return 30
    if nombre == 'zacatecas':
        return 31
    if nombre == 'importación': 
        return 32

In [12]:
adj_2019['source']=adj_2019['origen'].apply(to_id)
adj_2019['tarjet']=adj_2019['destino'].apply(to_id)
edges_2019 = pd.DataFrame()
edges_2019['source']=adj_2019['origen'].apply(to_id)
edges_2019['tarjet']=adj_2019['destino'].apply(to_id)
edges_2019['freq']=adj_2019['frecuencia']

edges_2019.head()

,source,tarjet,freq
1,1,6,6
2,1,10,24
3,1,16,12
4,1,18,18
5,1,28,6


In [14]:
edges_2019.info()
#vemos que no haya valores nulos

<class 'pandas.core.frame.DataFrame'>
Int64Index: 353 entries, 1 to 383
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   source  353 non-null    int64
 1   tarjet  353 non-null    int64
 2   freq    353 non-null    int64
dtypes: int64(3)
memory usage: 11.0 KB


# hasta aquí jala sin graph-tool

In [ ]:
g = gt.Graph()   
freq = g.new_ep('int')
g.add_edge_list(edges_2019.values, eprops=[freq]) 

In [ ]:
g

In [ ]:
coordenadas

In [ ]:
pos = gt.arf_layout(g)

In [ ]:
for i in range(0,33):
    pos[g.vertex(i)] = (coordenadas.values[i-1,2], coordenadas.values[i-1,1]*-1)
 
list(pos)

In [ ]:
gt.graph_draw(g, pos=pos, edge_color=freq)